---
**Titre** : Cross-Modalité & Réunion Précoce  
**Auteur** : Dmitrašinović Théotime  
**Date** : 07/11/2023  
  
Il existe plusieurs méthodes de réunification des modalités:
1. ***Réunion précoce***
2. Réunion tardive
3. Réunion intermédiaire  
  
**But** :  
Dans ce NoteBook nous allons traiter la ***Réunion Précoce***.    
Il s'agit de réunir les caractéristiques avant la Classification.  
Pour ce faire plusieurs concaténations des vecteurs des 3 modalités sont réalisées.


**Etapes**:
1. Charger les caractéristiques des différentes modalités
2. Réunir les caractéristiques de différentes manières
3. Classifier la réunion des caractéristiques

*Des classes python sont définies pour faciliter les combinaisons.*


---

## Google Drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px

# Les Différentes Labelisations

In [ ]:
# DataFrame des individus labélisés
CSVLabelisedDataPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/indice_vid_labelled.csv"
df_labels = pd.read_csv(CSVLabelisedDataPath)

# Labels du papier prédis par un modèle de language à partir de la transcription
CSV_NLPP = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLPP.csv"
df_NLPP = pd.read_csv(CSV_NLPP, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})
CSV_NLPS = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLPS.csv"
df_NLPS = pd.read_csv(CSV_NLPS, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})
CSV_NLP_A1k = "/content/gdrive/MyDrive/Projet_Multimedia/download/CSV/NLP_A1k.csv"
df_NLP_A1k = pd.read_csv(CSV_NLP_A1k, index_col=False)[['name', 'new_label']].rename(columns={'new_label':'label'})

In [ ]:
# On supprime une vidéo car non présente dans les embeddings vidéo
df_labels.drop(df_labels[df_labels['name'] == '-_YoeHOTJBI4'].index, inplace=True)

# Les Caractéristiques  
Elles sont toutes enregistrées sous un format dictionnaire (la clé est l'identifiant de la vidéo).   
Je les charge en DataFrame pour manipulation simplifiée

## AUDIO  
**moyenne et max_pool VGGish & proba moyenne YAMNet**  
et combinaisons...

In [ ]:
# Les embeddings Audio
# VGGish
AverageEmbeddingsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/VGGish/embeddings_average_VGGish.pkl"
MaxPoolEmbeddingsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/VGGish/embeddings_max_pooling_VGGish.pkl"
# YAMNet
AveragePredictionsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Audio_Embeddings/YAMNet/ALL_predictions_YAMNet.pkl"

def getEmbesDF(path):
  with open(path, "rb") as fp:
      embeddings = pickle.load(fp)
  df_embes = pd.DataFrame.from_dict((embeddings), orient='index')
  return df_embes

# Les 3 classiques
df_avg_embes = getEmbesDF(AverageEmbeddingsPath)
df_max_pool_embes = getEmbesDF(MaxPoolEmbeddingsPath)
df_avg_preds = getEmbesDF(AveragePredictionsPath)

# Différentes Concaténations des 3 vecteurs (avg_embes, max_pool_embes et avg_preds)
# VGGish max_pool & VGGish avg embes
df_max_pool_et_avg_embes = df_avg_embes.join(df_max_pool_embes.rename(columns={k:1000+k for k in df_max_pool_embes.columns}))
# VGGish max_pool & YAMNet avg predictions
df_max_pool_et_avg_preds = df_avg_preds.join(df_max_pool_embes.rename(columns={k:1000+k for k in df_max_pool_embes.columns}))
# VGGish max_pool & VGGish avg embes & YAMNet avg predictions
df_VGGish_et_YAMNet = df_max_pool_et_avg_embes.join(df_avg_preds.rename(columns={k:2000+k for k in df_avg_preds.columns}))
# VGGish max_pool + VGGish avg embes
df_addition_VGGish = df_avg_embes + df_max_pool_embes
# YAMNet avg predictions & (VGGish max_pool + VGGish avg embes)
df_addition_VGGish_et_YAMNet = df_addition_VGGish.join(df_avg_preds.rename(columns={k:1000+k for k in df_avg_preds.columns}))

# DICO RECAPITULATIF AVEC LEGENDE
AUDIO_EMBES = {"Average VGGish":df_avg_embes,
               "Max_pool VGGish":df_max_pool_embes,
               "Average Environment Sound":df_avg_preds,
               "VGGish max_pool & VGGish avg embes":df_max_pool_et_avg_embes,
               "VGGish max_pool & YAMNet avg predictions":df_max_pool_et_avg_preds,
               "VGGish max_pool & VGGish avg embes & YAMNet avg predictions":df_VGGish_et_YAMNet,
               "VGGish max_pool + VGGish avg embes":df_addition_VGGish,
               "YAMNet avg predictions & (VGGish max_pool + VGGish avg embes)":df_addition_VGGish_et_YAMNet
               }

### prints

In [ ]:
df_avg_embes

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
-AZS5cgybKcI,-0.284029,0.032755,0.318786,-0.508630,-0.158374,-0.537420,-0.272469,-0.247939,-0.489918,-0.421043,...,-0.750526,0.275619,0.006835,-0.046609,0.047074,0.087500,-0.190523,-0.319955,0.313330,-0.104647
-uSTNyHkde08,-0.151631,0.126783,0.009415,-0.808152,-0.167872,-0.080102,-0.276894,-0.163420,-0.419045,-0.328050,...,-0.845795,0.426596,-0.251827,-0.044944,0.011121,-0.121800,0.044923,0.382124,0.337639,-0.112781
-lvK2TPBDvl4,-0.098594,-0.052492,-0.110585,-0.009614,0.013139,-0.133419,-0.167707,-0.178166,-0.333955,-0.176828,...,-0.369322,0.103515,-0.143293,0.129249,-0.105774,-0.048344,0.353801,0.061696,0.225369,0.045655
-OT8lw1Rl-WM,-0.333148,-0.167473,0.068600,1.011651,-0.096846,-0.342634,-0.379832,0.075960,-0.371182,-0.500207,...,-0.173641,0.186639,-0.098925,-0.267119,-0.397092,-0.148772,0.067599,-0.249984,-0.066723,-0.206089
-u9hauSrihYQ,0.079281,-0.028590,-0.016004,-0.681075,0.094331,-0.406786,-0.167358,-0.368698,-0.669619,-0.474807,...,-1.293930,0.272565,-0.074230,0.400362,-0.154907,-0.062980,0.478865,0.338979,0.541409,-0.225782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-XxhKUP9Ixco,-0.280628,0.091947,0.259500,-0.605304,-0.178549,-0.792526,-0.340907,-0.269865,-0.669721,-0.457272,...,-0.838962,0.269498,-0.099941,0.156100,0.040538,-0.033792,0.015712,-0.143638,0.299129,-0.117889
-9qSEfcIfYbw,-0.228944,0.236101,0.189116,-0.778044,-0.325997,-0.896316,-0.307132,-0.384035,-0.699013,0.107635,...,-1.113585,0.374020,-0.108004,-0.182393,0.128016,0.119758,-0.155420,-0.331021,0.156119,-0.113735
-JM1pDskO-mM,0.247100,-0.197765,0.088442,-0.657029,-0.047162,-0.215567,-0.292102,-0.414455,-0.509692,-0.180268,...,-0.911134,0.193714,-0.081714,0.085447,-0.319141,0.143900,0.109137,0.561742,0.259562,-0.171799
-by1s5NZZcpc,-0.769470,-0.088049,0.204124,-0.812171,-0.209300,-0.458896,-0.181901,0.165653,-0.370192,-0.554795,...,-0.446702,0.913508,-0.209537,-0.064853,-0.011300,0.067624,-0.187202,-0.630378,0.430468,0.119026


In [ ]:
df_max_pool_embes

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
-AZS5cgybKcI,0.467929,1.158297,2.155282,0.884385,0.557746,0.332980,0.300339,0.497729,0.595764,0.664417,...,0.303603,1.691528,1.546429,0.625743,1.484951,0.921381,0.544346,0.398715,1.297890,0.782649
-uSTNyHkde08,0.415817,1.253828,0.983704,0.252166,1.234288,0.737561,0.362188,0.381105,0.677649,0.271813,...,0.109509,1.372819,0.363047,0.517522,1.176547,0.516852,1.518862,1.323369,1.150133,0.486254
-lvK2TPBDvl4,1.215612,0.575120,0.301489,0.295747,0.631949,0.490955,0.303432,0.281541,0.545851,0.294721,...,-0.092364,1.259574,0.236476,1.536513,0.337351,0.847221,2.661507,1.940617,1.297495,0.207542
-OT8lw1Rl-WM,-0.045211,0.108199,0.416568,2.021822,0.379551,-0.030114,-0.042822,0.385462,0.168607,-0.011783,...,0.422308,0.519035,0.113991,-0.013330,-0.030054,0.078741,0.392364,0.052072,0.228777,0.177694
-u9hauSrihYQ,0.961878,0.565337,0.772123,0.295747,0.828234,0.530017,0.494858,0.785376,0.846989,0.088328,...,-0.048727,0.936038,1.486860,1.314309,0.834661,0.857056,1.721961,1.252215,2.671279,1.141855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-XxhKUP9Ixco,0.738772,1.129693,1.445872,0.297412,0.514642,-0.045885,0.396317,0.772363,0.520450,0.416508,...,0.441539,1.017560,0.802700,1.438932,1.510991,0.589828,1.390938,1.654803,1.522970,0.540876
-9qSEfcIfYbw,0.208700,1.126496,1.082255,-0.315098,0.045243,-0.509598,0.184367,0.311139,1.423268,1.460236,...,-0.446488,1.472245,0.870158,0.290051,1.111118,0.435245,0.123261,0.089042,0.836138,0.422470
-JM1pDskO-mM,1.377999,0.666915,1.491539,-0.017875,0.611549,0.665041,0.515618,0.390662,0.725203,0.666959,...,0.141704,1.213070,0.803988,0.764159,0.560836,0.897992,0.650932,1.823366,1.317222,0.546739
-by1s5NZZcpc,-0.087133,0.633227,0.940148,0.299564,0.238056,0.028004,0.299984,1.125408,0.356890,0.267938,...,0.365244,2.680022,0.235550,0.458004,1.197537,1.067621,0.502374,0.114645,1.917000,0.923379


In [ ]:
df_avg_preds

,0,1,2,3,4,5,6,7,8,9,...,511,512,513,514,515,516,517,518,519,520
-AZS5cgybKcI,0.025266,0.001835,1.925003e-04,0.000397,1.446317e-04,0.000072,4.624372e-05,4.732015e-06,3.638251e-05,1.735417e-06,...,4.330092e-05,9.821250e-07,1.839629e-04,5.289807e-04,2.474351e-05,7.315379e-05,1.748715e-04,0.001129,0.000285,7.037500e-05
-uSTNyHkde08,0.901550,0.000252,1.308382e-03,0.016251,9.275483e-07,0.021930,2.495787e-05,8.117040e-07,2.121147e-05,4.280596e-07,...,1.431625e-05,1.749882e-06,5.454943e-05,3.374542e-04,1.035530e-04,3.252637e-05,1.793257e-04,0.000465,0.001299,5.271555e-06
-lvK2TPBDvl4,0.289680,0.000088,1.798792e-03,0.003571,4.857006e-08,0.000229,1.813735e-06,7.948588e-08,1.241068e-06,3.755787e-08,...,2.261237e-07,7.088843e-10,5.419458e-08,3.762583e-07,2.553923e-08,2.225540e-08,6.532983e-08,0.000013,0.000018,1.035225e-08
-OT8lw1Rl-WM,0.000165,0.000019,1.042128e-06,0.000027,1.152583e-09,0.000005,1.747444e-08,4.968363e-12,5.525666e-09,1.668432e-12,...,1.326842e-05,1.014781e-08,5.257566e-08,9.533384e-08,5.398527e-10,6.021600e-08,1.187875e-08,0.000007,0.000010,7.441545e-10
-u9hauSrihYQ,0.873961,0.000537,5.590712e-03,0.019910,2.669240e-05,0.000472,6.581184e-05,2.157791e-06,7.214768e-05,1.203631e-06,...,2.459629e-06,2.274370e-07,1.540807e-05,1.419609e-04,6.659506e-05,6.534895e-06,1.155913e-05,0.000170,0.000153,3.343296e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-7aXpBF2mjtg,0.000198,0.000004,6.313921e-07,0.000009,2.222242e-10,0.000005,9.458382e-06,3.512658e-08,3.595870e-06,9.238269e-08,...,2.260958e-05,7.437294e-10,3.508532e-06,3.261856e-07,2.932347e-10,4.832601e-07,3.888806e-07,0.003454,0.000101,4.476325e-07
-2REkk9SCRn0,0.016435,0.000499,1.121820e-04,0.000277,4.467005e-05,0.000045,6.285003e-05,3.551395e-06,5.236510e-05,2.508292e-06,...,7.062811e-06,2.063451e-06,3.393742e-05,1.259191e-04,4.296944e-06,8.703074e-03,2.638522e-05,0.001168,0.000189,4.278257e-05
-g4woIrtHTOc,0.503371,0.001570,2.650695e-03,0.013245,5.734666e-06,0.000275,6.412073e-06,3.957370e-08,3.304207e-06,3.486264e-08,...,5.720985e-06,1.885902e-08,1.986428e-08,2.641852e-07,1.540977e-08,9.366641e-08,1.067836e-07,0.000268,0.000078,7.340961e-08
-pQu3dufotPM,0.948938,0.000963,1.105974e-02,0.016475,1.078703e-05,0.000761,1.136992e-05,4.609679e-07,7.582129e-06,2.589666e-07,...,1.025157e-06,1.184329e-07,6.000910e-07,2.608112e-06,1.913772e-07,3.376322e-07,2.451204e-07,0.000166,0.000434,5.093843e-08


In [ ]:
df_avg_preds.loc[['-_YoeHOTJBI4']]

,0,1,2,3,4,5,6,7,8,9,...,511,512,513,514,515,516,517,518,519,520
-_YoeHOTJBI4,0.705247,0.004406,0.004065,0.00558,0.000959,0.000535,0.000412,0.000033,0.000323,0.000023,...,0.000009,4.613752e-07,0.000016,0.000032,0.000006,0.000012,0.00001,0.000644,0.000537,0.000005


In [ ]:
HH = df_labels.set_index('name').join(df_avg_preds)
HH.loc[['-8BbZTCdB2X4']]

,label,0,1,2,3,4,5,6,7,8,...,511,512,513,514,515,516,517,518,519,520
name,,,,,,,,,,,,,,,,,,,,,
-8BbZTCdB2X4,Entertainment,0.82355,0.002822,0.009282,0.023985,0.000033,0.006549,0.000012,2.195528e-07,0.000008,...,0.000005,0.000002,0.000001,0.000001,1.260052e-07,0.000002,1.841089e-07,0.000273,0.000217,1.624610e-07


In [ ]:
df_avg_preds.loc[['-8BbZTCdB2X4']]

,0,1,2,3,4,5,6,7,8,9,...,511,512,513,514,515,516,517,518,519,520
-8BbZTCdB2X4,0.82355,0.002822,0.009282,0.023985,0.000033,0.006549,0.000012,2.195528e-07,0.000008,7.931755e-08,...,0.000005,0.000002,0.000001,0.000001,1.260052e-07,0.000002,1.841089e-07,0.000273,0.000217,1.624610e-07


## TEXTE

In [ ]:
TexteEmbeddingsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Texte_Embeddings/"
# pretrained classifier
#df_DistilledBERT =
# complex
df_NPNet = getEmbesDF(TexteEmbeddingsPath + "all_embeddings_MPnet.pkl")
# classic
df_sentenceBERT = getEmbesDF(TexteEmbeddingsPath + "all_embeddings_sentenceBERT_on_Text.pkl")

# DICO RECAPITULATIF AVEC LEGENDE
TEXT_EMBES = {"NPNet":df_NPNet,
              }

In [ ]:
index_a_effacer = set(df_NPNet.index)-set(df_sentenceBERT.index)
len(index_a_effacer)

195

### prints

In [ ]:
df_NPNet

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
-KHQk1_Vq69E,-0.078369,0.104353,-0.035927,0.029662,0.027819,0.004518,0.061032,-0.061135,0.020897,-0.053106,...,0.100091,0.009520,-0.018339,0.045757,0.013256,0.007558,-0.034337,-0.026190,0.073885,-0.000449
-mG4Y2Snygfk,-0.040944,0.005626,-0.015000,-0.027005,-0.004517,0.054473,-0.036581,-0.001693,-0.004708,-0.016208,...,0.003352,0.030587,-0.030484,0.002562,-0.031805,0.020068,-0.008954,0.033752,0.046760,0.017519
-OdEfx8up7wM,0.024012,0.012302,0.012738,0.011366,-0.011972,0.028367,-0.039290,-0.001809,-0.075600,-0.021770,...,0.006665,0.012087,0.046532,-0.023250,0.009190,0.034071,0.029253,0.031046,0.000417,-0.037190
-Opbfh4wNu7Q,0.000454,0.067417,0.009428,-0.025054,-0.011213,-0.004910,0.002500,-0.008905,0.032295,0.024868,...,-0.003031,0.008893,0.052106,0.021486,-0.006882,-0.107162,0.041598,0.061760,-0.075058,0.011580
-A5340xbbQJU,0.003017,-0.046368,0.016738,-0.000405,0.025040,0.028105,0.037504,0.009014,-0.042572,-0.036270,...,-0.019537,0.048102,-0.095373,-0.015992,0.011701,0.035589,-0.020492,0.021927,-0.001086,0.021300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-ymc30meWzfg,0.027108,0.019392,0.001783,-0.054240,0.034475,0.039292,-0.003054,0.037774,-0.056808,0.005527,...,0.021888,-0.018802,-0.031078,0.037964,-0.047328,-0.007568,0.019388,0.018097,0.046122,-0.063908
-h7HGWRTGqyg,0.024681,-0.017745,0.006444,0.009110,-0.027458,-0.006089,0.006903,-0.016782,-0.050325,0.013674,...,-0.000614,0.056968,-0.012382,-0.009581,-0.014625,0.046420,0.010481,0.020612,-0.005490,-0.015180
-XVRLTfN1WYw,0.000065,0.051108,-0.014925,-0.035909,0.065775,0.022429,-0.041913,0.001468,-0.016216,0.028514,...,-0.051669,-0.005461,0.021186,0.010780,-0.011465,-0.025083,0.004139,0.017765,-0.059931,-0.025393
-jTtH7Fq1trw,0.021409,-0.019283,-0.016048,0.016259,-0.031077,-0.019049,-0.033932,0.044088,-0.077190,0.044629,...,-0.042829,0.025547,-0.027205,-0.030817,0.006751,0.056108,-0.022057,0.024049,0.030772,-0.015636


In [ ]:
df_sentenceBERT

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
-dXyrxjIcWdw,-0.075608,0.062685,-0.202902,0.063869,-0.247141,0.364194,-0.123600,0.365860,-0.075920,-0.221617,...,-0.392125,-0.067659,0.252899,0.209545,-0.232329,0.162588,-0.168940,0.519111,-0.251271,0.054707
-g0FMrjsYAzc,-0.224170,0.197426,-0.096950,0.075534,0.093223,-0.183442,-0.182965,-0.482303,-0.185558,-0.220866,...,0.013301,0.295778,0.257401,-0.004663,-0.292113,-0.197606,0.302074,0.326714,0.133684,0.054002
-h7HGWRTGqyg,-0.184037,0.206572,0.239960,0.276989,-0.092446,-0.075209,-0.138990,0.535187,0.353225,0.137935,...,-0.487990,0.080708,-0.026105,0.278784,-0.359980,0.377781,-1.065270,0.289604,-0.024068,-0.463253
-eOrNdBpGMv8,0.003118,0.109009,0.073999,-0.229716,-0.049716,-0.076584,-0.127622,-0.295880,0.005240,-0.017874,...,-0.484302,0.112567,0.147130,-0.226871,-0.623133,-0.105636,0.019261,0.040174,0.045401,-0.250675
-dRaKMNYG3-I,-0.162758,0.006307,0.038191,-0.261744,0.141425,-0.154015,-0.072139,0.708730,-0.244871,0.017090,...,-0.278745,0.160994,0.028104,0.140059,-0.801814,0.235661,-0.058569,0.228247,0.350008,-0.529724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
--UggNH1YeRY,0.003775,0.008079,-0.106113,0.474609,0.240930,-0.187067,0.166127,0.404034,-0.120493,0.088942,...,-0.380016,0.177476,0.508867,0.065690,0.314269,0.059263,0.098556,0.387254,-0.154260,-0.212655
--kNSaU8L3mI,-0.108383,0.437879,0.128701,0.132936,-0.117360,0.216841,0.064894,0.226753,0.251363,0.701871,...,-0.097925,-0.031245,0.039698,-0.045187,0.645504,0.180559,-0.902727,0.758704,-0.227333,-0.095497
-0-NBRA1aSXk,-0.210953,0.338499,0.153246,0.145230,-0.301000,0.315762,0.054305,0.019658,0.242963,0.199082,...,-0.341005,0.208765,0.536749,0.414233,-0.012642,0.069974,0.022425,-0.075114,-0.263534,0.224236
--bCmuE3KxWI,0.061514,0.218632,-0.121800,0.487341,-0.162928,-0.578505,0.096635,0.038756,-0.311475,0.367859,...,-0.279752,0.014966,0.559802,0.334159,-0.087687,0.154619,-0.089980,0.505168,0.191813,-0.367363


## VIDEO

In [ ]:
VideoEmbeddingsPath = "/content/gdrive/MyDrive/Projet_Multimedia/download/Embeddings/Video_Embeddings/"
#YOLO
# moyenne
df_YOLO_avgProba = getEmbesDF(VideoEmbeddingsPath + "all_embeddings_avgProba.pkl")
df_YOLO_avgProba.rename(columns={k:i for i,k in enumerate(df_YOLO_avgProba.columns)}, inplace=True)
# max
df_YOLO_maxProba = getEmbesDF(VideoEmbeddingsPath + "all_embeddings_maxProba.pkl")
df_YOLO_maxProba.rename(columns={k:i for i,k in enumerate(df_YOLO_maxProba.columns)}, inplace=True)
# sentence BERT
df_YOLO_sentenceBERT = getEmbesDF(VideoEmbeddingsPath + "all_embeddings_sentenceBERT.pkl")

# DICO RECAPITULATIF AVEC LEGENDE
VIDEO_EMBES = {"YOLO_avgProba":df_YOLO_avgProba,
               "YOLO_maxProba":df_YOLO_maxProba,
               "YOLO_sentenceBERT":df_YOLO_sentenceBERT
               }

### prints

In [ ]:
df_YOLO_avgProba

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
-AZS5cgybKcI,0.502143,0.315217,0.430597,0.522195,0.676364,0.346439,0.470266,0.365000,0.461429,0.580208,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
-XRqE7nagL_E,1.273582,0.537766,0.417397,0.876932,0.865310,0.434472,0.466567,0.365000,0.438000,0.578724,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
-IrJ4rmE2zuY,1.273582,0.537766,0.408706,0.905963,0.865310,0.414124,0.466567,0.362895,0.380000,0.578724,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
-OSKNiwkaRRo,1.273582,0.537766,0.408706,0.927743,0.865310,0.420085,0.466567,0.362895,0.377917,0.578724,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
-LTXS17y6Cz8,1.273582,0.537766,0.408706,0.925981,0.865310,0.420085,0.466567,0.362895,0.377917,0.578724,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-W2Cxs_p2-oE,0.822802,0.489905,0.471855,1.151121,0.653521,0.534871,0.523058,0.419909,0.715845,0.658056,...,0.50029,0.519059,0.493117,0.426825,0.477513,0.460594,0.543528,0.474957,0.44004,0.379932
-dKXpAC311Lg,0.822802,0.489905,0.471855,1.151121,0.653521,0.534871,0.523058,0.419909,0.715845,0.658056,...,0.50029,0.519059,0.493117,0.426825,0.477513,0.460594,0.543528,0.474957,0.44004,0.379932
-WQ258MgqdEo,0.822802,0.489905,0.471855,1.151133,0.653521,0.534865,0.523051,0.419909,0.715837,0.658056,...,0.50029,0.519059,0.493117,0.426825,0.477513,0.460594,0.543528,0.474957,0.44004,0.379932
-CzZ5Q9w217I,0.822799,0.489905,0.471855,1.150982,0.653521,0.534856,0.523051,0.419888,0.715837,0.657930,...,0.50029,0.519059,0.493090,0.426825,0.477513,0.460830,0.543528,0.474957,0.44004,0.379932


In [ ]:
df_YOLO_maxProba

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
-AZS5cgybKcI,1.26,0.45,0.83,2.37,0.91,0.60,1.05,0.67,0.70,1.35,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-XRqE7nagL_E,2.82,0.78,0.83,2.70,0.94,0.87,1.05,0.67,0.70,1.35,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-IrJ4rmE2zuY,2.82,0.78,0.83,2.70,0.94,0.87,1.05,0.67,0.70,1.35,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-OSKNiwkaRRo,2.82,0.78,0.83,3.41,0.94,1.19,1.05,0.67,0.70,1.35,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
-LTXS17y6Cz8,2.82,0.78,0.83,3.41,0.94,1.19,1.05,0.67,0.70,1.35,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-W2Cxs_p2-oE,7.47,3.03,3.26,11.21,3.40,4.76,2.94,1.56,12.63,3.98,...,1.73,3.75,1.88,2.17,2.59,3.25,2.67,2.02,0.76,0.75
-dKXpAC311Lg,7.47,3.03,3.26,11.21,3.40,4.76,2.94,1.56,12.63,3.98,...,1.73,3.75,1.88,2.17,2.59,3.25,2.67,2.02,0.76,0.75
-WQ258MgqdEo,7.47,3.03,3.26,11.21,3.40,4.76,2.94,1.56,12.63,3.98,...,1.73,3.75,1.88,2.17,2.59,3.25,2.67,2.02,0.76,0.75
-CzZ5Q9w217I,7.47,3.03,3.26,11.21,3.40,4.76,2.94,1.56,12.63,3.98,...,1.73,3.75,1.88,2.17,2.59,3.25,2.67,2.02,0.76,0.75


In [ ]:
df_YOLO_sentenceBERT

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
-KHQk1_Vq69E,0.408733,-0.285013,1.013274,-0.123601,-0.020243,-0.182481,-0.204677,0.816266,0.167600,-0.091708,...,-0.049126,-0.505683,0.359784,0.340195,0.089724,0.318646,0.263765,0.096415,0.092599,-0.083568
-mG4Y2Snygfk,-0.008985,-0.108422,0.337404,-0.285643,0.837284,-0.300680,-0.134623,0.369804,-0.223383,-0.253825,...,-0.046637,-0.016035,0.297653,-0.012892,0.444077,0.110807,0.042282,-0.030162,-0.434790,-0.006410
-OdEfx8up7wM,0.170914,0.101618,0.020646,-0.227854,-0.104497,0.394797,0.124065,0.287620,-0.244088,-0.179582,...,0.022679,-0.084234,-0.088674,-0.284039,0.191425,0.135168,-0.601380,-0.056669,0.168430,-0.067118
-Opbfh4wNu7Q,0.142487,-0.027481,-0.013731,0.052390,0.391897,0.256630,-0.108599,0.050855,0.232441,-0.388174,...,0.360538,-0.052663,-0.250196,-0.521948,0.167736,0.341699,0.208047,-0.422181,-0.417172,0.045517
-A5340xbbQJU,0.214984,0.345481,-0.018829,-0.319107,-0.450214,-0.195176,0.074612,0.126444,-0.299925,-0.390381,...,0.153401,0.017706,0.062022,0.196087,0.581182,0.372682,-0.283173,0.281638,-0.105126,-0.006993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-ymc30meWzfg,0.058945,0.407970,-0.004589,-0.066186,0.404911,0.183303,-0.119138,-0.019039,-0.111550,-0.247691,...,0.381052,0.070516,0.001582,-0.050510,0.247958,0.057875,-0.329769,-0.060673,-0.076438,-0.071345
-h7HGWRTGqyg,-0.002229,-0.325732,-0.068398,0.033980,0.142067,0.092510,0.061620,0.667728,-0.304149,0.147238,...,-0.311507,0.064851,0.011647,-0.166361,-0.077442,-0.071569,-0.209433,-0.292913,-0.199101,-0.037413
-XVRLTfN1WYw,-0.022620,0.047745,-0.147563,0.240614,-0.237503,0.002725,0.097245,0.308877,0.197665,0.022798,...,0.338123,-0.076645,0.043697,-0.382231,0.153545,0.155630,-0.161847,0.158812,-0.158440,-0.168552
-jTtH7Fq1trw,0.025042,-0.305272,-0.138818,0.062167,-0.101963,0.075844,-0.078886,0.518659,-0.023669,-0.282229,...,-0.305901,0.066279,0.059586,0.027975,0.021466,0.126443,0.104615,0.038750,-0.020270,-0.023294


# Les différentes concaténations des caractéristiques

In [ ]:
def concat3DF(audioDF, videoDF, textDF):
  # nom de colonne arbitraire
  return audioDF.join(videoDF.rename(columns={k:5000+k for k in videoDF.columns})).join(textDF.rename(columns={k:10000+k for k in textDF.columns}))

In [ ]:
print("Voici les différentes combinaisons des caractéristiques des 3 modalités:")
CONCA_EMBES = {}
for audio_cara in AUDIO_EMBES:
  for video_cara in VIDEO_EMBES:
    for text_cara in TEXT_EMBES:
      txt = 'A:'+audio_cara+ ' |V:'+video_cara+ ' |T:'+text_cara
      print(txt)
      CONCA_EMBES[txt] = concat3DF(AUDIO_EMBES[audio_cara], VIDEO_EMBES[video_cara], TEXT_EMBES[text_cara])

Voici les différentes combinaisons des caractéristiques des 3 modalités:
A:Average VGGish |V:YOLO_avgProba |T:NPNet
A:Average VGGish |V:YOLO_maxProba |T:NPNet
A:Average VGGish |V:YOLO_sentenceBERT |T:NPNet
A:Max_pool VGGish |V:YOLO_avgProba |T:NPNet
A:Max_pool VGGish |V:YOLO_maxProba |T:NPNet
A:Max_pool VGGish |V:YOLO_sentenceBERT |T:NPNet
A:Average Environment Sound |V:YOLO_avgProba |T:NPNet
A:Average Environment Sound |V:YOLO_maxProba |T:NPNet
A:Average Environment Sound |V:YOLO_sentenceBERT |T:NPNet
A:VGGish max_pool & VGGish avg embes |V:YOLO_avgProba |T:NPNet
A:VGGish max_pool & VGGish avg embes |V:YOLO_maxProba |T:NPNet
A:VGGish max_pool & VGGish avg embes |V:YOLO_sentenceBERT |T:NPNet
A:VGGish max_pool & YAMNet avg predictions |V:YOLO_avgProba |T:NPNet
A:VGGish max_pool & YAMNet avg predictions |V:YOLO_maxProba |T:NPNet
A:VGGish max_pool & YAMNet avg predictions |V:YOLO_sentenceBERT |T:NPNet
A:VGGish max_pool & VGGish avg embes & YAMNet avg predictions |V:YOLO_avgProba |T:NPNet


In [ ]:
CONCA_EMBES['A:Average VGGish |V:YOLO_avgProba |T:NPNet']

,0,1,2,3,4,5,6,7,8,9,...,10758,10759,10760,10761,10762,10763,10764,10765,10766,10767
-AZS5cgybKcI,-0.284029,0.032755,0.318786,-0.508630,-0.158374,-0.537420,-0.272469,-0.247939,-0.489918,-0.421043,...,-0.016345,0.011871,0.081454,0.028434,0.000720,0.048081,-0.019239,0.017322,-0.005079,-0.010430
-uSTNyHkde08,-0.151631,0.126783,0.009415,-0.808152,-0.167872,-0.080102,-0.276894,-0.163420,-0.419045,-0.328050,...,0.052252,-0.030660,-0.070046,0.016502,0.018954,0.026817,0.012793,-0.021116,0.021013,-0.005806
-lvK2TPBDvl4,-0.098594,-0.052492,-0.110585,-0.009614,0.013139,-0.133419,-0.167707,-0.178166,-0.333955,-0.176828,...,-0.007566,-0.012531,-0.053592,-0.031459,-0.020901,0.032501,-0.017626,0.023362,0.010696,0.011437
-OT8lw1Rl-WM,-0.333148,-0.167473,0.068600,1.011651,-0.096846,-0.342634,-0.379832,0.075960,-0.371182,-0.500207,...,-0.042251,0.003278,0.013946,-0.016444,0.030174,0.002022,-0.010580,0.015989,-0.028080,0.028416
-u9hauSrihYQ,0.079281,-0.028590,-0.016004,-0.681075,0.094331,-0.406786,-0.167358,-0.368698,-0.669619,-0.474807,...,0.034831,0.007491,-0.033891,0.006666,0.028671,-0.015324,-0.027003,0.064464,0.024482,-0.021678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-XxhKUP9Ixco,-0.280628,0.091947,0.259500,-0.605304,-0.178549,-0.792526,-0.340907,-0.269865,-0.669721,-0.457272,...,-0.032628,0.017044,-0.054296,0.010720,-0.013955,0.011555,-0.037130,0.009045,-0.046849,-0.008343
-9qSEfcIfYbw,-0.228944,0.236101,0.189116,-0.778044,-0.325997,-0.896316,-0.307132,-0.384035,-0.699013,0.107635,...,-0.036809,0.005251,0.060685,0.047007,-0.025875,0.003642,0.009807,0.039794,-0.026634,-0.004177
-JM1pDskO-mM,0.247100,-0.197765,0.088442,-0.657029,-0.047162,-0.215567,-0.292102,-0.414455,-0.509692,-0.180268,...,0.026697,0.009370,0.050035,-0.040253,0.004477,-0.050364,-0.009875,0.009346,-0.022041,0.000307
-by1s5NZZcpc,-0.769470,-0.088049,0.204124,-0.812171,-0.209300,-0.458896,-0.181901,0.165653,-0.370192,-0.554795,...,-0.047370,0.010925,0.037121,0.000555,0.022265,0.000275,0.002496,0.016271,-0.042304,0.008178


### Fonctions

In [ ]:
# CORPUS
class Corpus:
  #df_labels = pd.read_csv(CSVLabelisedDataPath)
  def __init__(self, embeddings, separation, l, s, c, df_labels):
    self.df_labels = df_labels
    self.nom = '||Label:' + l + " ||Separation: " + s + " ||CARA: " + c
    self.nom_sep = s
    self.nom_label = l
    self.nom_cara = c

    self.embeddings = embeddings
    # separation
    self.train, self.test, self.valid = separation(self.df_labels.copy(deep=True))
    # jointure avec les embeddings
    self.join()
    # Séparation de X et y
    self.Xy()

  def join(self):
    self.train = self.train.set_index('name').join(self.embeddings)
    self.test = self.test.set_index('name').join(self.embeddings)
    self.valid = self.valid.set_index('name').join(self.embeddings)

  def Xy(self):
    self.Xtrain = self.train.drop(columns=['label'])
    self.ytrain = self.train.label
    self.Xtest = self.test.drop(columns=['label'])
    self.ytest = self.test.label
    self.Xvalid = self.valid.drop(columns=['label'])
    self.yvalid = self.valid.label

# CLASSIF
class Classif:

  def __init__(self, corpus, nom_model, accuracy, predictions):
    self.corpus = corpus
    self.nom = self.corpus.nom + '|' + nom_model
    self.nom_model = nom_model
    self.accuracy = accuracy
    self.predictions = predictions

# FONCTIONS
def split_TTV(df):
  # Split the data into train, test, and validation sets while maintaining the class distribution
  # Adjust the test_size and random_state parameters as needed
  train, temp = train_test_split(df, train_size=0.6, stratify=df.label, random_state=42)
  test, val = train_test_split(temp, test_size=0.5, stratify=temp.label, random_state=42)

  # You can access the subsets like this
  #print("Training set:", train.shape)
  #print("Testing set:", test.shape)
  #print("Validation set:", val.shape)
  return train, test, val

#Séparation 1
def not_balanced(df):
  return split_TTV(df)

#Séparation 2
def delHalfEduc(df_labels):
  # on supprime la moitié des individus avec label Education car sûr représentés

  subset_to_del = df_labels[df_labels['label']=="Education"].sample(len(df_labels[df_labels['label']=="Education"])//2)
  df_labels = df_labels.drop(subset_to_del.index)
  return split_TTV(df_labels)
#Séparation 2
def delHalfAccess(df_labels):
  # on supprime la moitié des individus avec label Education car sûr représentés

  subset_to_del = df_labels[df_labels['label']=="accessibility"].sample(len(df_labels[df_labels['label']=="accessibility"])//2)
  df_labels = df_labels.drop(subset_to_del.index)
  return split_TTV(df_labels)


#Séparation 3
# SurSampling x2 et x3
def surSamplingBasic(df, seuil=4):
  vc = df['label'].value_counts()
  sx2 = int(np.ceil(len(df)*0.02))
  lx2 = vc[vc < sx2].index
  sx3 = int(np.ceil(len(df)*0.05))
  lx3 = vc[vc < sx3].index
  # créer une copie du df auquel on rajoute des nouveaux individus
  balanced_df = df.copy(deep=True)
  for l in lx2:
    label_data = df[df['label'] == l]
    sx2 = min(len(label_data)*2, sx2)
    to_add_subset = label_data.sample(sx2-len(label_data), replace=True)
    balanced_df = pd.concat([balanced_df, to_add_subset])
  for l in lx3:
    label_data = df[df['label'] == l]
    sx3 = min(len(label_data)*3, sx3)
    to_add_subset = label_data.sample(sx3-len(label_data), replace=True)
    balanced_df = pd.concat([balanced_df, to_add_subset])
  return balanced_df
def sursamplingx2x3(df, seuil=0.05):
  train, test, val = split_TTV(df)
  train = surSamplingBasic(train, seuil)
  test = surSamplingBasic(test, seuil)
  valid = surSamplingBasic(val, seuil)
  return train, test, valid

#Séparation 4
# SurSampling et SousSampling
def SurEtSousSampling(df):
  # on supprime la moitié des individus avec label Education car sûr représentés
  train, test, valid = delHalfEduc(df)
  train = surSamplingBasic(train, 0.05)
  test = surSamplingBasic(test, 0.05)
  valid = surSamplingBasic(valid, 0.05)
  return train, test, valid

#Séparation 5
# changement de label pour les classes minoritaires
def redistribution(df):
  switch_label = {"Gaming":"Entertainment", "Travel & Events":"People & Blogs", "Nonprofits & Activism": "News & Politics"}
  true_Voiture_label = {"-AwRAfxBub9M":"Film & Animation",
                        "-1VM2eLhvsSM":"Film & Animation",
                        "-DfPMxdHZKsw":"People & Blogs",
                        "-XsVV0t_cS7Q":"People & Blogs",
                        "-GHDz-XDD8OU":"Film & Animation",
                        "-PXJnZMF3ucc":"People & Blogs",
                        "-CeN6PMEOQEA":"People & Blogs",
                        "-q6rAllJAdWk":"People & Blogs",
                        "-jHCA59ia_NE":"People & Blogs",
                        "-VsJKPdOuzZw":"Comedy"}
  for old_label in switch_label:
    df.loc[df.label==old_label, ['label']] = switch_label[old_label]
  for id in true_Voiture_label:
    df.loc[df.name==id, ['label']] = true_Voiture_label[id]
  # on delete Autos & Vehicles
  df.drop(df[df['label']=="Autos & Vehicles"].index, inplace=True)
  return df
def redisEtSurSous(df):
  df = redistribution(df)
  return SurEtSousSampling(df)



from sklearn.metrics import confusion_matrix
def show_conf_matrix(y_test, pred):
  list_cat= list(set(y_test))
  conf_matrix = confusion_matrix(y_test, pred, labels=list_cat)
  cf = [[i/sum(ligne) for i in ligne] for ligne in conf_matrix]
  data = cf
  fig = px.imshow(data,
                  labels=dict(y="Vrai Catégorie", x="Catégorie prédite", color="Productivity"),
                  x=list_cat,
                  y=list_cat
                )
  fig.update_xaxes(side="top")
  fig.show()

# Les différents modèles de Classification

In [ ]:
# pipeline & normalisation
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# SVC & OneVsRest & NaiveBayes
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB

MODELS = {'SVC':make_pipeline(StandardScaler(), SVC(gamma='auto')),
          '1vsRestSVC':make_pipeline(StandardScaler(), OneVsRestClassifier(SVC(gamma='auto'))),
          'Naive Bayes':make_pipeline(StandardScaler(),GaussianNB())
          }

In [ ]:
# LABELS
CSV_labels = {"Labels Normaux":df_labels, "NLP Proba":df_NLPP, "NLP scricte":df_NLPS}
CSV_labels = {"Labels A1k":df_NLP_A1k}
CSV_labels = {"Labels Normaux":df_labels}

# METHODES DE SEPARATION DU DATASET
Separations = {"Not balanced":not_balanced,
               "Halfed education":delHalfEduc,
               "Sur Sample x2x3":sursamplingx2x3,
               "Sur & Sous Sample":SurEtSousSampling,
               "Redistribution Small labels & Sur Sous Sample":redisEtSurSous}



# Constructions de toutes les combinaisons de Séparations et Caractéristiques extraites de l'Audio avec potentielement labels différents
List_Corpus = []
for l in CSV_labels:
  for s in Separations:
    for c in CONCA_EMBES:
      List_Corpus.append(Corpus(embeddings=CONCA_EMBES[c], separation=Separations[s], l=l, s=s, c=c, df_labels=CSV_labels[l]))
      print(List_Corpus[-1].nom)
print("\n-", len(CSV_labels)*len(Separations)*len(CONCA_EMBES), "Combinaisons possibles pour lesquelles on va essayer chaque modèle de Classification")

||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average VGGish |V:YOLO_avgProba |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average VGGish |V:YOLO_maxProba |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average VGGish |V:YOLO_sentenceBERT |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Max_pool VGGish |V:YOLO_avgProba |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Max_pool VGGish |V:YOLO_maxProba |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Max_pool VGGish |V:YOLO_sentenceBERT |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average Environment Sound |V:YOLO_avgProba |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average Environment Sound |V:YOLO_maxProba |T:NPNet
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average Environment Sound |V:YOLO_sentenceBERT |T:NPNet
||Label:Labels Normaux ||Separation: Not ba

In [ ]:
from tqdm.notebook import tqdm_notebook

# tous les corpus et tous les modèles
List_Classif = []
for corpus in tqdm_notebook(List_Corpus, desc='Corpus'):
  for nom_model in tqdm_notebook(MODELS, desc='Models', leave=False):
    model = MODELS[nom_model]
    model.fit(corpus.Xtrain, corpus.ytrain)
    predictions = model.predict(corpus.Xtest)
    accuracy = model.score(corpus.Xtest, corpus.ytest)
    List_Classif.append(Classif(corpus, nom_model, accuracy, predictions))

Corpus:   0%|          | 0/120 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

...

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(len(List_Classif))
for i, classif in enumerate(List_Classif):
  if classif.accuracy > 0.50:
    print(classif.nom)
    print("accuracy:",classif.accuracy)
    show_conf_matrix(classif.corpus.ytest, classif.predictions)

135
||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average VGGish |V:YOLO_avgProba |T:NPNet|1vsRestSVC
accuracy: 0.5394736842105263


||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average VGGish |V:YOLO_maxProba |T:NPNet|1vsRestSVC
accuracy: 0.5372807017543859


||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average VGGish |V:YOLO_sentenceBERT |T:NPNet|1vsRestSVC
accuracy: 0.506578947368421


||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Max_pool VGGish |V:YOLO_avgProba |T:NPNet|1vsRestSVC
accuracy: 0.5263157894736842


||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Max_pool VGGish |V:YOLO_maxProba |T:NPNet|1vsRestSVC
accuracy: 0.5219298245614035


||Label:Labels Normaux ||Separation: Not balanced ||CARA: A:Average Environment Sound |V:YOLO_avgProba |T:NPNet|1vsRestSVC
accuracy: 0.5043859649122807


||Label:Labels Normaux ||Separation: Halfed education ||CARA: A:Max_pool VGGish |V:YOLO_sentenceBERT |T:NPNet|1vsRestSVC
accuracy: 0.5013054830287206
